# Robust Scaler

This is a component that scale features using statistics that are robust to outliers. This Scaler removes the median and scales the data according to the quantile range (defaults to IQR: Interquartile Range). The IQR is the range between the 1st quartile (25th quantile) and the 3rd quartile (75th quantile). It makes use of an implementation from [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html). 
<br>
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

This notebook shows:
- how to use the [SDK](https://platiagro.github.io/sdk/) to load datasets, save models and other artifacts.
- how to declare parameters and use them to build reusable components.

## Declare parameters
Components may declare (and use) these default parameters:
- dataset
- target
- experiment_id
- operator_id

Use these parameters to load/save datasets, models, metrics, and figures with the help of [PlatIAgro SDK](https://platiagro.github.io/sdk/).

You may also declare custom parameters to set when running an experiment.

In [ ]:
dataset = "iris" #@param {type:"string"}
target = "Species" #@param {type:"string"}
experiment_id = "94c3e6b9-0420-4d48-a5df-2d31fc2ad3af" #@param {type:"string"}
operator_id = "a43611a0-3d85-4bee-b94a-4ab7b01e9e21" #@param {type:"string"}

## Load dataset

Import and put the whole dataset in a pandas.DataFrame.

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)
X = df.drop(target, axis=1).to_numpy()
y = df[target].to_numpy()

In [ ]:
import numpy as np

columns = df.columns.to_numpy()
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)

## Load metadata about the dataset
For example, below we get the feature type for each column in the dataset. (eg. categorical, numerical, or datetime)

In [ ]:
from platiagro import stat_dataset
from platiagro.featuretypes import infer_featuretypes

try:
    metadata = stat_dataset(name=dataset)
    featuretypes = metadata["featuretypes"]
except KeyError:
    featuretypes = infer_featuretypes(df)

featuretypes = np.array(featuretypes)
featuretypes = np.delete(featuretypes, target_index)

## Fit a model using sklearn.preprocessing.RobustScaler

In [ ]:
import pandas as pd
from platiagro.featuretypes import NUMERICAL
from sklearn.preprocessing import RobustScaler

# selects the indexes of numerical features
numerical_indexes = (featuretypes == NUMERICAL)

estimator = RobustScaler()

if np.ma.any(numerical_indexes):
    X[:, numerical_indexes] = estimator.fit_transform(X[:, numerical_indexes])

    # Put data back in a pandas.DataFrame
    df = pd.DataFrame(data=X, columns=columns)
    df[target] = pd.Series(y)

## Save dataset

Stores the transformed dataset in a object storage.<br>

In [ ]:
from platiagro import save_dataset

save_dataset(name=dataset, df=df)

## Save model

Stores the model artifacts in a object storage.<br>
It will make the model available for future deployments.

In [ ]:
from platiagro import save_model

save_model(experiment_id=experiment_id,
           model={"estimator": estimator,
                  "columns": columns.tolist(),
                  "numerical_indexes": numerical_indexes})